<a href="https://colab.research.google.com/github/gmsharpe/edumore/blob/master/terraform_aws_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Terraform, AWS Athena & Google Colab

## Download and Install Terraform

In [ ]:
%%bash
# instructions from https://learn.hashicorp.com/tutorials/terraform/install-cli?in=terraform/aws-get-started
sudo apt-get update && sudo apt-get install -y gnupg software-properties-common curl
curl -fsSL https://apt.releases.hashicorp.com/gpg | sudo apt-key add -
sudo apt-add-repository "deb [arch=amd64] https://apt.releases.hashicorp.com $(lsb_release -cs) main"
sudo apt-get update && sudo apt-get install terraform

## Download and Install the AWS CLI

In [ ]:
%%bash

# download the installation file
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"

unzip -q awscliv2.zip

sudo ./aws/install


You can now run: /usr/local/bin/aws --version


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 43.4M  100 43.4M    0     0  23.1M      0  0:00:01  0:00:01 --:--:-- 23.0M



## Verify the Installation of the AWS CLI

In [ ]:
%%bash
aws -version


usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help

aws: error: the following arguments are required: command



## Configure the AWS Client with your Credentials

Using `aws configure`, you next need to set your credentials and defaults for the AWS CLI.

For more detailed instructions, see [Quick configuration with aws configure](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-quickstart.html).

In [ ]:
# importing os module
import os
  
path = "/content/terraform_tutorial"
  
if not os.path.exists(path):
  os.mkdir(path)

os.chdir(path)
main_tf = open("main.tf","w")
variables_tf = open("variables.tf","w")

variables = """
locals {
  database_s3_bucket_name = "yellow-taxi-ny-983700"
  query_results_s3_bucket_name = "yellow-taxi-ny-983700-workgroup-query-results"
}
"""

main_body = """
terraform {
  required_providers {
    aws = {
      source  = "hashicorp/aws"
      version = "~> 3.27"
    }
  }
  required_version = ">= 0.14.9"
}

provider "aws" {
  profile = "default"
  region  = "us-west-1"
}

resource "aws_s3_bucket" "bucket" {
  bucket = "${local.database_s3_bucket_name}"
}

# Ensure no public access is possible
resource "aws_s3_bucket_public_access_block" "bucket" {
  bucket = aws_s3_bucket.bucket.id

  block_public_acls       = true
  block_public_policy     = true
  ignore_public_acls      = true
  restrict_public_buckets = true
}
"""

main_tf.write(main_body)
main_tf.close()
variables_tf.write(variables)
variables_tf.close()

In [ ]:
%%bash
terraform init 


Initializing the backend...

Initializing provider plugins...
- Finding hashicorp/aws versions matching "~> 3.27"...
- Installing hashicorp/aws v3.71.0...
- Installed hashicorp/aws v3.71.0 (signed by HashiCorp)

Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


In [ ]:
%%bash
cd terraform_tutorial
terraform plan

In [ ]:
%%bash
terraform apply -auto-approve

In [ ]:
%%bash
wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2021-01.csv -o /dev/null

In [ ]:
%%bash
aws s3 cp /content/terraform_tutorial/yellow_tripdata_2021-01.csv s3://yellow-taxi-ny-983700
aws s3 ls s3://yellow-taxi-ny-983700/

upload: ./yellow_tripdata_2021-01.csv to s3://yellow-taxi-ny-983794/yellow_tripdata_2021-01.csv
2022-01-08 02:10:55          0 cb7da54e-d080-456f-bedd-4787eb3124a6.txt
2022-01-09 00:03:25  125981363 yellow_tripdata_2021-01.csv
